#Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# LLMs Lab-1

# Accessing OpenAI LLM through an API

In [ ]:
!pip install OpenAI
!pip install langchain
!pip install langchain_community
!pip install Cohere
!pip install langchain-openai langchain-cohere python-dotenv

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
from google.colab import userdata


In [ ]:
openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key


In [ ]:
#from langchain.llms import OpenAI, Cohere
from langchain_openai import OpenAI
llm= OpenAI()


In [ ]:
print(llm.invoke("How many minimum sleeping hours you should have?"))

In [ ]:
print(llm.invoke("What is the sentiment of this review. Give one word answer Pos or Neg - The staff was friendly and helpful. I had a great experience with my loan officer."))

### creating list and converting to dataframe

In [ ]:
import pandas as pd

# Create a list of log messages
log_messages = [
    "Server started successfully",
    "Low disk space",
    "Database connection failed",
    "Security breach detected",
    "New feature released",
]

# Create a DataFrame with 10 rows and 1 column
df = pd.DataFrame(log_messages, columns=["Message"])

In [ ]:
# Classify each log message using llm.invoke
df["Sentiment"] = df["Message"].apply(lambda message: llm.invoke(f"Classify the severity of this log message make give output as CRITICAL if the message is critical: {message}"))

In [ ]:
df

### reading from csv file and getting sentiments

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/giridhar276/Datasets/refs/heads/main/moviesentiments/movie_reviews.csv")

In [ ]:
df

In [ ]:
# Classify each log message using llm.invoke
df["Sentiment"] = df["review"].apply(lambda message: llm.invoke(f"Classify the sentiment of this  message make give output as positive or negative only: {message}"))

In [ ]:
df

### specifying required model in openai

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [ ]:

resp = llm.invoke("How many minimum sleeping hours should an adult have?")
print(resp.content)

### using cohere

In [ ]:

cohere_key = userdata.get("COHERE_API_KEY")
os.environ["COHERE_API_KEY"] = cohere_key

In [ ]:
#from langchain.llms import OpenAI, Cohere
from langchain_cohere import ChatCohere
llm= ChatCohere()


In [ ]:
print(llm.invoke("How many minimum sleeping hours you should have?").content)

In [ ]:
print(llm.invoke("What is the sentiment of this review. Give one word answer Pos or Neg - The staff was friendly and helpful. I had a great experience with my loan officer.").content)

### creating list and converting to dataframe reading from it

In [ ]:
import pandas as pd

# Create a list of log messages
log_messages = [
    "Server started successfully",
    "Low disk space",
    "Database connection failed",
    "Security breach detected",
    "New feature released",
]

# Create a DataFrame with 10 rows and 1 column
df = pd.DataFrame(log_messages, columns=["Message"])

In [ ]:
df

### reading from csv file and getting sentiments

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/giridhar276/Datasets/refs/heads/main/moviesentiments/movie_reviews.csv")

In [ ]:
df

In [ ]:
# Classify each log message using llm.invoke
df["Sentiment"] = df["review"].apply(lambda message: llm.invoke(f"Classify the sentiment of this  message make give output as positive or negative only: {message}"))

In [ ]:
df

### specifying model in coher

In [ ]:
from langchain_cohere import ChatCohere
cllm = ChatCohere(model="command-r-plus", temperature=0.2)
print(cllm.invoke("Answer in one line: minimum adult sleep hours?").content)

### batch with concurrency (LangChain)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd



In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/giridhar276/Datasets/refs/heads/main/moviesentiments/movie_reviews.csv")


In [ ]:
# Keep the instruction out of the loop; send only the variable text per row
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a strict sentiment classifier. "
               "Return exactly 'positive' or 'negative' only."),
    ("human", "Text: {text}\nLabel:")
])
chain = prompt | llm


In [ ]:

texts = df["review"].astype(str).tolist()
texts


In [ ]:

# Batch requests; tune max_concurrency based on your rate limits (e.g., 5–10)
outs = chain.batch([{"text": t} for t in texts], config={"max_concurrency": 8})


In [ ]:

labels = [o.content.strip().lower() for o in outs]
df["Sentiment"] = ["positive" if l.startswith("pos") else "negative" for l in labels]


In [ ]:
df

In [ ]:
def classify(txt):
    resp = llm.invoke(
        "You are a strict sentiment classifier. "
        "Return exactly 'positive' or 'negative'.\n\n"
        f"Text: {txt}\nLabel:"
    )
    label = resp.content.strip().lower()
    return "positive" if label.startswith("pos") else "negative"

sentiments = []
for review in df["review"].astype(str).tolist():
    sentiments.append(classify(review))

df["Sentiment"] = sentiments

### same as above with progress bar

In [ ]:
from tqdm.auto import tqdm

def classify(txt):
    resp = llm.invoke(
        "You are a strict sentiment classifier. "
        "Output exactly 'positive' or 'negative' only.\n\n"
        f"Text: {txt}\nLabel:"
    )
    label = resp.content.strip().lower()
    return "positive" if label.startswith("pos") else "negative"

sentiments = []
df = df.iloc[0:3]
for review in tqdm(df["review"].astype(str).tolist()):
    sentiments.append(classify(review))

df["Sentiment"] = sentiments

In [ ]:
df

# Gen AI - Code Generation in Google Colab

Import dataset from https://raw.githubusercontent.com/giridhar276/Datasets/master/Automobile%20Data%20Set/AutoDataset.csv

print number of rows and columns

Print the column names

Create a bar chart for the 'fuel-type' column

Findout outliers in price column